In [1]:
from pathlib import Path

import wandb
import torch
from lightning.pytorch import loggers, utilities, Trainer

from model import Digits
from data_module import MNISTDataModule

In [ ]:
root_path = Path('../')
dm = MNISTDataModule(data_dir=(root_path / 'data').as_posix())

In [3]:
log_dir = root_path / 'logs'
logger = loggers.WandbLogger(
    project='digits',
    name='evaluation',
    save_dir=log_dir,
    log_model='all',
)

artifact_dir = root_path / 'artifacts'
artifact_dir.mkdir(exist_ok=True)
model_path = Path(loggers.WandbLogger.download_artifact(
    artifact='sampath017/digits/model-vtm5y8me:v43',
    artifact_type='model',
    save_dir=artifact_dir
))

model_path = model_path / 'model.ckpt'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sampath017. Use `wandb login --relogin` to force relogin


CommError: Unable to fetch artifact with name sampath017/digits/model-vtm5y8me:v43

In [ ]:
model = Digits.load_from_checkpoint(model_path, map_location=torch.device('cpu'))
utilities.model_summary.ModelSummary(model)

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 322 K 
-------------------------------------
322 K     Trainable params
0         Non-trainable params
322 K     Total params
1.288     Total estimated model params size (MB)

In [ ]:
trainer = Trainer(
    inference_mode=True,
    logger=logger,
    enable_model_summary=False
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.test(model, datamodule=dm);

Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     99.13999938964844     │
│         test_loss         │   0.026899944990873337    │
└───────────────────────────┴───────────────────────────┘

In [ ]:
wandb.finish()

epoch,▁▁▁
test_acc,▁▁▁
test_loss,▁▁▁
trainer/global_step,▁▁▁
epoch,0
test_acc,99.14
test_loss,0.0269
trainer/global_step,0
